In [26]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
# Load raw text
text = open('tinyshakespeare.txt','r').read()

In [8]:
# Get the unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [43]:
# encoders and decoders
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [44]:
# encode the text
data = torch.tensor(encode(text), dtype=torch.long)

In [45]:
# split the data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [46]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [47]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_simaxze):
  context = x[:t+1]
  target = y[t]
  print(f"{context} => {target}")

tensor([18]) => 47
tensor([18, 47]) => 56
tensor([18, 47, 56]) => 57
tensor([18, 47, 56, 57]) => 58
tensor([18, 47, 56, 57, 58]) => 1
tensor([18, 47, 56, 57, 58,  1]) => 15
tensor([18, 47, 56, 57, 58,  1, 15]) => 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) => 58


In [48]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb = get_batch('train')
print(f"inputs: {xb.shape}\n{xb}")
print(f"targets: {yb.shape}\n{yb}")
print("--------------")

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"{context} => {target}")

inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------------
tensor([24]) => 43
tensor([24, 43]) => 58
tensor([24, 43, 58]) => 5
tensor([24, 43, 58,  5]) => 57
tensor([24, 43, 58,  5, 57]) => 1
tensor([24, 43, 58,  5, 57,  1]) => 46
tensor([24, 43, 58,  5, 57,  1, 46]) => 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) => 39
tensor([44]) => 53
tensor([44, 53]) => 56
tensor([44, 53, 56]) => 1
tensor([44, 53, 56,  1]) => 58
tensor([44, 53, 56,  1, 58]) => 46
tensor([44, 53, 56,  1, 58, 46]) => 39
tensor([44, 53, 56,  1, 58, 46, 39]) => 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) => 1
tensor([52]) => 58
tensor([52, 58]) => 1
tensor([52, 58,  1]) => 58


In [49]:
class BigramLanguageModel(nn.Module):
  
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
  def forward(self, idx, targets=None):
    
    logits = self.token_embedding_table(idx)
    
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    
    return logits, loss
  
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx
  
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

idx = torch.zeros((1,1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

lfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


In [50]:
# PyTorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [53]:
batch_size = 32
for steps in range(10000):
  
  xb,yb = get_batch('train')
  
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  
print(loss.item())

2.3645713329315186


In [55]:
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


Seve bed, ld OROR:
Dores,
IOMyoninon.
O, corspappand ber's then t, la cks boceme,


Foure ys!
Manin t atheresue mo-OMIS:
Y:
JUCI y fe das withondo!
SSheamind RYofouso neer wal,
OMAUCar,
VOred wharso mers adeatharllencaze trts Y brds be oy.
K: t men I be anthechy the ont

blinithterurighe pld y,
De omoo besintht hincr he t t norttoy, thod. po ham tslel

Bim ord couecor INo, thangh sermncello t y, f


In [60]:

B,T,C = 4,8,2
x = torch.randn(B,T,C)

#x[b,t] = mean over i <= t of x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev, 0)

In [61]:
xbow[0]

tensor([[-0.9966, -1.1894],
        [-0.2304, -0.7437],
        [ 0.1511, -0.1415],
        [ 0.1348,  0.1488],
        [ 0.0854,  0.4298],
        [ 0.0167,  0.1611],
        [ 0.0374,  0.0828],
        [-0.0412,  0.2217]])

In [62]:
x[0]

tensor([[-0.9966, -1.1894],
        [ 0.5359, -0.2980],
        [ 0.9140,  1.0630],
        [ 0.0860,  1.0196],
        [-0.1125,  1.5539],
        [-0.3264, -1.1824],
        [ 0.1614, -0.3868],
        [-0.5914,  1.1934]])

In [64]:
(-0.9+0.5)/2

-0.2

In [105]:
mat = torch.tril(torch.ones(8,8))/torch.tril(torch.ones(8,8)).sum(1).view(-1,1)

In [107]:
mat @ x[0]

tensor([[-0.9966, -1.1894],
        [-0.2304, -0.7437],
        [ 0.1511, -0.1415],
        [ 0.1348,  0.1488],
        [ 0.0854,  0.4298],
        [ 0.0167,  0.1611],
        [ 0.0374,  0.0828],
        [-0.0412,  0.2217]])

In [130]:
# self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [131]:
k.transpose(-2,-1).shape

torch.Size([4, 16, 8])

In [132]:
k.shape

torch.Size([4, 8, 16])

In [133]:
wei.var()

tensor(0.0287, grad_fn=<VarBackward0>)

In [134]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [135]:
q.var()

tensor(0.3386, grad_fn=<VarBackward0>)

In [141]:
k = torch.randn((B,T,16))
q = torch.randn((B,T,16))
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [142]:
k.var()

tensor(1.0037)

In [143]:
q.var()

tensor(1.0966)

In [144]:
wei.var()

tensor(1.2769)